In [26]:
import pandas as pd
import numpy as np
from pprint import pprint

In [27]:
#import the dataset
training_data = pd.read_csv("project1 1.data",header=None, names=['price','maint','doors','persons','lug_boot','safety','result'])
testing_data = pd.read_csv("project1_test.data",header=None, names=['price','maint','doors','persons','lug_boot','safety','result'])
training_data.head()

,price,maint,doors,persons,lug_boot,safety,result
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


# Information Gain Based Decision Tree Model

In [28]:
###defining entropy
def entropy(target_col):
    #using unique ,getting the number  unique elements and elements
    elements,counts = np.unique(target_col,return_counts=True)
    #formula
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy
  
##Info Gain

def InfoGain(data,split_attribute_name,target_name="result"):
    #calculate total entropy
    total_entropy = entropy(data[target_name])
    #get individual values
    vals,counts = np.unique(data[split_attribute_name],return_counts=True)
    #cal the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).
                                dropna()[target_name])for i in range(len(vals))])
    
    #formula for information gain
    Information_Gain = total_entropy-Weighted_Entropy
    return Information_Gain

In [29]:
def create_decision_tree(data,originaldata,features,target_attribute_name="result",parent_node_class=None):
    #If all target_values have the same value,return this value
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    #if the dataset is empty
    elif len(data) == 0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],
                                                                           return_counts=True)[1])]
    
    #If the feature space is empty
    elif len(features) == 0:
        return parent_node_class 

    #If none of the above condition holds true grow the tree

    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],
                                                                           return_counts=True)[1])]

    #Select the feature which best splits the dataset
    item_values = [InfoGain(data,feature,target_attribute_name)for feature in features] #Return the infgain values
    best_feature_index = np.argmax(item_values)
    best_feature = features[best_feature_index]

    #Create the tree structure
    tree = {best_feature:{}}

    #Remve the feature with the best info gain
    features = [i for i in features if i!= best_feature]

    #Grow the tree branch under the root node

    for value in np.unique(data[best_feature]):
        value = value
        sub_data = data.where(data[best_feature]==value).dropna()
        #call the ID3 algotirthm
        subtree = create_decision_tree(sub_data,data,features,target_attribute_name,parent_node_class)
        #Add the subtree
        tree[best_feature][value] = subtree
    return(tree)

# Accuracy Functions

In [30]:
#Predict
def predict(query,tree,default=1):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
               result = tree[key][query[key]]
            except:
               return default

            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result

In [31]:
##check the accuracy
def test(data,tree):
   queries = data.iloc[:,:-1].to_dict(orient="records")
   predicted = pd.DataFrame(columns=["predicted"])
    
   #calculation of accuracy

   for i in range(len(data)):
       predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0)
   
   print("The Prediction accuracy is:",(np.sum(predicted["predicted"]==data["result"])/len(data))*100,'%')

# Result

In [32]:
#Train the tree,print the tree abnd predict the accuracy
tree = create_decision_tree(training_data,training_data,training_data.columns[:-1])
pprint(tree)

{'safety': {'high': {'persons': {2.0: 'unacc',
                                 4.0: {'price': {'high': {'maint': {'high': 'acc',
                                                                    'low': 'acc',
                                                                    'med': 'acc',
                                                                    'vhigh': 'unacc'}},
                                                 'low': {'maint': {'high': {'lug_boot': {'big': 'vgood',
                                                                                         'med': {'doors': {2.0: 'acc',
                                                                                                           3.0: 'acc',
                                                                                                           4.0: 'vgood',
                                                                                                           6.0: 'vgood'}},
                               

In [24]:
test(testing_data,tree)


The Prediction accuracy is: 100.0 %


<h1><center>END</center></h1>